# Microsoft Professional Program for Data Science 

#### This notebook shall help preparing the data for the regression training by estimating all missing columns of incomplete data entries.

In [1]:
# Import standard library set and shared definitions of file names and paths
%run -i MPPSharedDefines.py
%matplotlib inline

In [6]:
training_data_pd = pd.read_csv(TEMP_PATH+CLEANED_OUTPUT_FN)
training_data_pd.head(10)

,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,applicant_ethnicity,applicant_race,applicant_sex,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant,rate_spread,sex,ethnicity,race,loan_type_name,property_type_name,purpose_name
0,0,2,1,1,1,139.0,1,261,14,246,2,5,1,77.0,5949.0,11.642,63192.0,100.000,1556.0,1993.0,2094,True,1.0,Male (1),Non-Hispanic(2),White(5),FHA-insured(2),1-4-family-non-mf(1),Home purchase(1)
1,1,2,2,1,1,160.0,3,349,48,311,1,5,1,42.0,2667.0,37.141,63693.0,75.729,875.0,1163.0,4194,False,2.0,Male (1),Hispanic/Latino(1),White(5),FHA-insured(2),Manufactured housing(2),Home purchase(1)
2,2,1,2,1,1,168.0,3,385,30,256,3,2,1,34.0,6393.0,79.635,42883.0,76.948,636.0,1611.0,1119,False,3.0,Male (1),Unknown(3),Asian(2),Conventional(1),Manufactured housing(2),Home purchase(1)
3,3,1,2,1,1,57.0,3,117,30,46,1,6,1,37.0,7175.0,72.843,63806.0,78.836,1587.0,2172.0,1119,False,6.0,Male (1),Hispanic/Latino(1),Unknown,Conventional(1),Manufactured housing(2),Home purchase(1)
4,4,2,1,2,1,25.0,3,95,41,192,2,5,2,46.0,5041.0,23.188,70908.0,59.579,756.0,1992.0,1593,False,4.0,Female (2),Non-Hispanic(2),White(5),FHA-insured(2),1-4-family-non-mf(1),Home improvement(2)
5,5,2,1,1,1,88.0,2,261,50,92,2,5,2,53.0,3278.0,4.913,55945.0,100.000,1141.0,2647.0,2800,True,1.0,Female (2),Non-Hispanic(2),White(5),FHA-insured(2),1-4-family-non-mf(1),Home purchase(1)
6,6,2,1,1,1,345.0,3,80,9,232,1,5,1,90.0,3827.0,16.331,114291.0,67.921,1113.0,1352.0,3797,True,1.0,Male (1),Hispanic/Latino(1),White(5),FHA-insured(2),1-4-family-non-mf(1),Home purchase(1)
7,7,2,1,1,1,123.0,3,189,3,207,1,5,1,38.0,5975.0,47.466,73584.0,52.528,1033.0,1766.0,2428,False,1.0,Male (1),Hispanic/Latino(1),White(5),FHA-insured(2),1-4-family-non-mf(1),Home purchase(1)
8,8,1,2,1,1,68.0,3,128,33,150,1,6,1,35.0,7294.0,24.169,57170.0,90.570,2078.0,2790.0,1268,True,2.0,Male (1),Hispanic/Latino(1),Unknown,Conventional(1),Manufactured housing(2),Home purchase(1)
9,9,1,2,1,1,27.0,3,261,29,119,2,5,1,39.0,6331.0,4.057,47600.0,96.910,1515.0,2436.0,1268,True,8.0,Male (1),Non-Hispanic(2),White(5),Conventional(1),Manufactured housing(2),Home purchase(1)


In [8]:
validation_data_pd = pd.read_csv(DATA_PATH+TEST_VALUES_FN)
validation_data_pd.head(10)

,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,applicant_ethnicity,applicant_race,applicant_sex,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant
0,0,1,2,1,1,46.0,3,262,37,83,1,5,2,50.0,10761.0,43.415,61732.0,100.000,2900.0,3603.0,1119,False
1,1,1,1,3,1,170.0,3,282,11,267,2,5,1,155.0,2285.0,20.115,50373.0,100.000,539.0,873.0,4216,False
2,2,1,1,1,1,174.0,3,274,12,162,1,5,1,102.0,7765.0,78.621,75175.0,54.165,1559.0,1712.0,2467,False
3,3,1,1,1,2,116.0,2,254,37,282,2,5,1,51.0,3387.0,96.208,48677.0,74.476,938.0,784.0,2773,True
4,4,1,1,3,1,108.0,3,194,41,17,2,5,1,88.0,3106.0,6.370,68070.0,100.000,1119.0,1301.0,3110,False
5,5,1,1,1,1,53.0,3,215,11,39,2,3,1,26.0,6338.0,89.621,68957.0,88.605,1303.0,1837.0,356,False
6,6,1,1,3,1,175.0,3,294,31,101,2,5,1,55.0,4488.0,4.468,84698.0,100.000,1290.0,1444.0,1657,False
7,7,2,1,1,1,135.0,2,339,14,230,1,5,1,40.0,6896.0,22.123,65203.0,81.272,1933.0,2772.0,809,False
8,8,2,1,1,1,290.0,3,29,30,174,1,5,1,72.0,4587.0,90.579,59894.0,100.000,884.0,1173.0,4243,False
9,9,1,1,1,1,4.0,3,154,33,232,2,5,1,57.0,7436.0,52.746,65855.0,88.008,1892.0,3039.0,3038,False


In [9]:
validation_data_pd.count()

row_id                            200000
loan_type                         200000
property_type                     200000
loan_purpose                      200000
occupancy                         200000
loan_amount                       200000
preapproval                       200000
msa_md                            200000
state_code                        200000
county_code                       200000
applicant_ethnicity               200000
applicant_race                    200000
applicant_sex                     200000
applicant_income                  189629
population                        198082
minority_population_pct           198080
ffiecmedian_family_income         198095
tract_to_msa_md_income_pct        198054
number_of_owner-occupied_units    198067
number_of_1_to_4_family_units     198067
lender                            200000
co_applicant                      200000
dtype: int64

In [14]:
# Receive training data target labels
training_rate_spread = np.array(training_data_pd['rate_spread'])

In [27]:
def remove_extended_columns(data_set):
    """
    Removes all columns which are not part of the test data
    """
    reduced_data_set = data_set.iloc[:,:22] # Take first 22 columns
    return reduced_data_set

pre_prepared_training_set = remove_extended_columns(training_data_pd)
pre_prepared_training_set.head(5)

,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,applicant_ethnicity,applicant_race,applicant_sex,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant
0,0,2,1,1,1,139.0,1,261,14,246,2,5,1,77.0,5949.0,11.642,63192.0,100.000,1556.0,1993.0,2094,True
1,1,2,2,1,1,160.0,3,349,48,311,1,5,1,42.0,2667.0,37.141,63693.0,75.729,875.0,1163.0,4194,False
2,2,1,2,1,1,168.0,3,385,30,256,3,2,1,34.0,6393.0,79.635,42883.0,76.948,636.0,1611.0,1119,False
3,3,1,2,1,1,57.0,3,117,30,46,1,6,1,37.0,7175.0,72.843,63806.0,78.836,1587.0,2172.0,1119,False
4,4,2,1,2,1,25.0,3,95,41,192,2,5,2,46.0,5041.0,23.188,70908.0,59.579,756.0,1992.0,1593,False


In [29]:
def convert_categorical_columns(data_set):
    """
    Removes all columns which are not part of the test data
    """
    reduced_data_set = data_set[['loan_amount', 'population', 'applicant_income', 'ffiecmedian_family_income', 
                                     'minority_population_pct', 'tract_to_msa_md_income_pct', 'number_of_owner-occupied_units',
                                    'number_of_1_to_4_family_units']] 
    return reduced_data_set

prepared_training_set = convert_categorical_columns(pre_prepared_training_set)
prepared_training_set.head(5)

,loan_amount,population,applicant_income,ffiecmedian_family_income,minority_population_pct,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units
0,139.0,5949.0,77.0,63192.0,11.642,100.000,1556.0,1993.0
1,160.0,2667.0,42.0,63693.0,37.141,75.729,875.0,1163.0
2,168.0,6393.0,34.0,42883.0,79.635,76.948,636.0,1611.0
3,57.0,7175.0,37.0,63806.0,72.843,78.836,1587.0,2172.0
4,25.0,5041.0,46.0,70908.0,23.188,59.579,756.0,1992.0


In [30]:
prepared_training_set.count()

loan_amount                       199783
population                        197801
applicant_income                  189112
ffiecmedian_family_income         197811
minority_population_pct           197801
tract_to_msa_md_income_pct        197773
number_of_owner-occupied_units    197784
number_of_1_to_4_family_units     197780
dtype: int64